# INFORMACIÓN DEL PROYECTO: JORGE ALONSO EDA

<div style="text-align: center;">
  <img src="./recursos/Automobile-Data.jpg" alt="Gráfico de ventas" width="700" />
</div>

### TÍTULO

**Estudio de datos almacenados por estaciones de carga para vehiculos eléctricos**

### TEMA

El tema principal es el estudio de los datos almacenados por puntos de recarga de coches eléctricos.

Para ello se va a utilizar el data set "Electric Vehicle Charging Patterns" estraido con una API de Kaggle. Los datos extraidos serán tratados y representados con el objetivo de estudar la forma en la que los conductores utilizan dichas estaciones de carga. Los datos extraidos son de conductores de EEUU durante los meses de enero y febrero de 2024. Las ciudades en las que se ubican los puntos de carga son: Houston, San Francisco, Los Angeles, Chicago y New York.

Los modelos de coches de os que se han recolectados los datos es de los 5 siguientes 5 vehículos: BMW i3, Hyundai Kona, Chevy Bolt, Nissan Leaf y Tesla Model 3. 


Este data set, se compone de un total de 20 columnas diferentes (cada una es un tipo de variable) y 1320 registros únicos de carga de EVs.

### HIPÓTESIS

Hipotesis 1: "Tesla es la marca más eficiente"<br>
Hipotesis 2: "Los cargadores de tipo DC Fast Charger son mas utilizados para distancias largas"<br>
Hipotesis 3: "Las ciudades con gran densidad de población, tienen mayor acceso a cargadores eléctricos, por lo que cargan más frecuentemente la batería del vehículo"

In [34]:
# Escribe tu hipótesis principal en Markdown
hipotesis_1 = "Tesla es la marca más eficient"

In [35]:
# Escribe más hipótesis si las hubiera en Markdown
# hipotesis_2 = "Tesla es la marca más eficiente"
# hipotesis_3 = ""
# hipotesis_4 = ""
# hipotesis_5 = ""
# ...


## OBTENCIÓN DE LOS DATOS

### DATASETS Y FUENTES ALTERNATIVAS DE DATOS

El siguiente data set sobre estaciones de carga de vehiculos eléctricos se muestra en bruto, es decir, se muestra tal cual ha sido descargado.

Durante el desarrollo del EDA se elavorará un proceso ETL, para poder sacar el máximo partido a este data set.

In [36]:
import hashlib
import requests
import datetime
import pandas as pd
import json
import kagglehub
from kagglehub import KaggleDatasetAdapter
import kaggle

In [37]:
#Descargar el data set de la api de kaggle

kaggle.api.authenticate()
kaggle.api.dataset_download_files("valakhorasani/electric-vehicle-charging-patterns", path="apiVehiculos", unzip=True)

Dataset URL: https://www.kaggle.com/datasets/valakhorasani/electric-vehicle-charging-patterns


In [38]:
ds_evs = pd.read_csv('./apiVehiculos/ev_charging_patterns.csv')

In [39]:
ds_evs.head()

,User ID,Vehicle Model,Battery Capacity (kWh),Charging Station ID,Charging Station Location,Charging Start Time,Charging End Time,Energy Consumed (kWh),Charging Duration (hours),Charging Rate (kW),Charging Cost (USD),Time of Day,Day of Week,State of Charge (Start %),State of Charge (End %),Distance Driven (since last charge) (km),Temperature (°C),Vehicle Age (years),Charger Type,User Type
0,User_1,BMW i3,108.463007,Station_391,Houston,2024-01-01 00:00:00,2024-01-01 00:39:00,60.712346,0.591363,36.389181,13.087717,Evening,Tuesday,29.371576,86.119962,293.602111,27.947953,2.0,DC Fast Charger,Commuter
1,User_2,Hyundai Kona,100.000000,Station_428,San Francisco,2024-01-01 01:00:00,2024-01-01 03:01:00,12.339275,3.133652,30.677735,21.128448,Morning,Monday,10.115778,84.664344,112.112804,14.311026,3.0,Level 1,Casual Driver
2,User_3,Chevy Bolt,75.000000,Station_181,San Francisco,2024-01-01 02:00:00,2024-01-01 04:48:00,19.128876,2.452653,27.513593,35.667270,Morning,Thursday,6.854604,69.917615,71.799253,21.002002,2.0,Level 2,Commuter
3,User_4,Hyundai Kona,50.000000,Station_327,Houston,2024-01-01 03:00:00,2024-01-01 06:42:00,79.457824,1.266431,32.882870,13.036239,Evening,Saturday,83.120003,99.624328,199.577785,38.316313,1.0,Level 1,Long-Distance Traveler
4,User_5,Hyundai Kona,50.000000,Station_108,Los Angeles,2024-01-01 04:00:00,2024-01-01 05:46:00,19.629104,2.019765,10.215712,10.161471,Morning,Saturday,54.258950,63.743786,203.661847,-7.834199,1.0,Level 1,Long-Distance Traveler


In [55]:
#Creamos la columna % total cargado por registro

ds_evs["% charged"] = ds_evs["State of Charge (End %)"] - ds_evs["State of Charge (Start %)"]
ds_evs

,User ID,Vehicle Model,Battery Capacity (kWh),Charging Station ID,Charging Station Location,Charging Start Time,Charging End Time,Energy Consumed (kWh),Charging Duration (hours),Charging Rate (kW),...,Day of Week,State of Charge (Start %),Temperature (°C),Vehicle Age (years),Charger Type,State of Charge (End %),User Type,% Charged,Distance Driven (since last charge) (km),% charged
0,User_1,BMW i3,108.463007,Station_391,Houston,2024-01-01 00:00:00,2024-01-01 00:39:00,60.712346,0.591363,36.389181,...,Tuesday,29.371576,27.947953,2.0,DC Fast Charger,86.119962,Commuter,56.748386,293.602111,56.748386
1,User_2,Hyundai Kona,100.000000,Station_428,San Francisco,2024-01-01 01:00:00,2024-01-01 03:01:00,12.339275,3.133652,30.677735,...,Monday,10.115778,14.311026,3.0,Level 1,84.664344,Casual Driver,74.548566,112.112804,74.548566
2,User_3,Chevy Bolt,75.000000,Station_181,San Francisco,2024-01-01 02:00:00,2024-01-01 04:48:00,19.128876,2.452653,27.513593,...,Thursday,6.854604,21.002002,2.0,Level 2,69.917615,Commuter,63.063011,71.799253,63.063011
3,User_4,Hyundai Kona,50.000000,Station_327,Houston,2024-01-01 03:00:00,2024-01-01 06:42:00,79.457824,1.266431,32.882870,...,Saturday,83.120003,38.316313,1.0,Level 1,99.624328,Long-Distance Traveler,16.504325,199.577785,16.504325
4,User_5,Hyundai Kona,50.000000,Station_108,Los Angeles,2024-01-01 04:00:00,2024-01-01 05:46:00,19.629104,2.019765,10.215712,...,Saturday,54.258950,-7.834199,1.0,Level 1,63.743786,Long-Distance Traveler,9.484836,203.661847,9.484836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315,User_1316,Nissan Leaf,100.000000,Station_57,New York,2024-02-24 19:00:00,2024-02-24 20:30:00,42.011654,1.426444,5.895475,...,Sunday,39.204102,1.919655,7.0,DC Fast Charger,83.915952,Commuter,44.711850,239.601075,44.711850
1316,User_1317,BMW i3,100.000000,Station_40,New York,2024-02-24 20:00:00,2024-02-24 20:44:00,68.185853,3.238212,18.388012,...,Tuesday,31.456375,34.029775,4.0,Level 2,93.096461,Casual Driver,61.640087,164.376022,61.640087
1317,User_1318,Nissan Leaf,100.000000,Station_374,New York,2024-02-24 21:00:00,2024-02-24 23:03:00,18.895102,3.267122,45.482066,...,Tuesday,71.903081,20.358761,5.0,DC Fast Charger,78.678879,Commuter,6.775798,226.519258,6.775798
1318,User_1319,Chevy Bolt,85.000000,Station_336,San Francisco,2024-02-24 22:00:00,2024-02-24 23:20:00,13.756252,2.754527,38.148183,...,Sunday,76.187997,24.134598,5.0,Level 2,65.926573,Commuter,-10.261424,291.494076,-10.261424


In [51]:
ds_evs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1320 entries, 0 to 1319
Data columns (total 21 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   User ID                                   1320 non-null   object 
 1   Vehicle Model                             1320 non-null   object 
 2   Battery Capacity (kWh)                    1320 non-null   float64
 3   Charging Station ID                       1320 non-null   object 
 4   Charging Station Location                 1320 non-null   object 
 5   Charging Start Time                       1320 non-null   object 
 6   Charging End Time                         1320 non-null   object 
 7   Energy Consumed (kWh)                     1254 non-null   float64
 8   Charging Duration (hours)                 1320 non-null   float64
 9   Charging Rate (kW)                        1254 non-null   float64
 10  Charging Cost (USD)                 

In [8]:
ds_evs["Vehicle Model"].unique()

array(['BMW i3', 'Hyundai Kona', 'Chevy Bolt', 'Nissan Leaf',
       'Tesla Model 3'], dtype=object)

In [9]:
ds_evs["Charging Station Location"].unique()

array(['Houston', 'San Francisco', 'Los Angeles', 'Chicago', 'New York'],
      dtype=object)

In [10]:
ds_evs["Charger Type"].unique()

array(['DC Fast Charger', 'Level 1', 'Level 2'], dtype=object)

In [15]:
ds_evs["Charging Station ID"].unique()

array(['Station_391', 'Station_428', 'Station_181', 'Station_327',
       'Station_108', 'Station_335', 'Station_162', 'Station_302',
       'Station_493', 'Station_452', 'Station_206', 'Station_78',
       'Station_16', 'Station_96', 'Station_147', 'Station_2',
       'Station_10', 'Station_434', 'Station_125', 'Station_81',
       'Station_485', 'Station_140', 'Station_326', 'Station_150',
       'Station_288', 'Station_361', 'Station_340', 'Station_129',
       'Station_438', 'Station_239', 'Station_58', 'Station_198',
       'Station_211', 'Station_402', 'Station_390', 'Station_45',
       'Station_97', 'Station_279', 'Station_23', 'Station_410',
       'Station_461', 'Station_44', 'Station_27', 'Station_282',
       'Station_463', 'Station_36', 'Station_109', 'Station_55',
       'Station_192', 'Station_219', 'Station_185', 'Station_69',
       'Station_317', 'Station_130', 'Station_378', 'Station_336',
       'Station_87', 'Station_294', 'Station_119', 'Station_208',
       'Stat

In [ ]:
#resumen de todas las stats principales en función de la ciudad

summary = ds_evs.groupby("Charging Station Location").describe()
print(summary)


                          Battery Capacity (kWh)                        \
                                           count       mean        std   
Charging Station Location                                                
Chicago                                    242.0  71.756381  22.125603   
Houston                                    262.0  73.722598  20.004793   
Los Angeles                                297.0  74.940816  21.523648   
New York                                   255.0  76.123589  19.447311   
San Francisco                              264.0  75.895798  19.727477   

                                                                         \
                                 min   25%   50%        75%         max   
Charging Station Location                                                 
Chicago                     3.838518  62.0  75.0  85.000000  188.634965   
Houston                     7.714517  62.0  75.0  85.000000  193.003074   
Los Angeles                 1.53